In [6]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import librosa.display
from scipy.ndimage import maximum_filter, gaussian_filter
import hashlib

In [ ]:


# Step 1: Load audio
y, sr = librosa.load('songs\\Run It Up - (Raag.Fm).mp3', sr=None)

# Step 2: Compute STFT
n_fft = 2048
hop_length = 512
S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
S_mag = np.abs(S)

# Step 3: Smooth spectrogram (optional, helps with noisy peaks)
S_smooth = gaussian_filter(S_mag, sigma=1.0)

# Step 4: 2D local maxima detection
neighborhood_size = 20
local_max = maximum_filter(S_smooth, size=neighborhood_size) == S_smooth

# Step 5: Apply a threshold to keep only strong peaks
threshold = np.percentile(S_smooth[local_max], 95)  # keep top 5%
peak_mask = local_max & (S_smooth >= threshold)

# Step 6: Get peak coordinates (freq_bin, time_bin)
peak_indices = np.argwhere(peak_mask)

# Step 7: Convert to time (sec) and frequency (Hz)
peak_freqs = peak_indices[:, 0] * (sr / n_fft)
peak_times = peak_indices[:, 1] * (hop_length / sr)

# Combine for (time, freq) tuples
constellation_map = list(zip(peak_times, peak_freqs))

# Print few peaks
print("Sample Peaks (time sec, frequency Hz):")
print(constellation_map[:10])

# Optional: Visualize peaks on spectrogram
"""plt.figure(figsize=(12, 6))
librosa.display.specshow(librosa.amplitude_to_db(S_mag, ref=np.max),
                         sr=sr, hop_length=hop_length, x_axis='time', y_axis='hz')
plt.scatter(peak_times, peak_freqs, marker='x', color='red', s=10, label='Peaks')
plt.title('Spectral Peaks (Constellation Map)')
plt.colorbar(format='%+2.0f dB')
plt.legend()
plt.show()"""

In [ ]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import librosa.display
from scipy.ndimage import maximum_filter, gaussian_filter

# Step 1: Load audio
y, sr = librosa.load('songs\\Run It Up - (Raag.Fm).mp3', sr=None)

# Step 2: Compute STFT
n_fft = 2048
hop_length = 512
S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
S_mag = np.abs(S)

# Step 3: Smooth spectrogram (optional, helps with noisy peaks)
S_smooth = gaussian_filter(S_mag, sigma=1.0)

# Step 4: 2D local maxima detection
neighborhood_size = 20
local_max = maximum_filter(S_smooth, size=neighborhood_size) == S_smooth

# Step 5: Apply a threshold to keep only strong peaks
threshold = np.percentile(S_smooth[local_max], 95)  # keep top 5%
peak_mask = local_max & (S_smooth >= threshold)

# Step 6: Get peak coordinates (freq_bin, time_bin)
peak_indices = np.argwhere(peak_mask)

# Step 7: Convert to time (sec) and frequency (Hz)
peak_freqs = peak_indices[:, 0] * (sr / n_fft)
peak_times = peak_indices[:, 1] * (hop_length / sr)

# Combine for (time, freq) tuples
constellation_map = list(zip(peak_times, peak_freqs))

# Print few peaks
print("Sample Peaks (time sec, frequency Hz):")
print(constellation_map[:10])

# Optional: Visualize peaks on spectrogram
"""plt.figure(figsize=(12, 6))
librosa.display.specshow(librosa.amplitude_to_db(S_mag, ref=np.max),
                         sr=sr, hop_length=hop_length, x_axis='time', y_axis='hz')
plt.scatter(peak_times, peak_freqs, marker='x', color='red', s=10, label='Peaks')
plt.title('Spectral Peaks (Constellation Map)')
plt.colorbar(format='%+2.0f dB')
plt.legend()
plt.show()"""

In [ ]:
import hashlib

def generate_hashes(peaks, fan_value=5):
    """
    Generate landmark hashes from peaks.
    Each hash is made from (freq1, freq2, delta_time), anchored at time1.
    """
    hashes = []
    peaks = sorted(peaks)  # Sort by time

    for i in range(len(peaks)):
        t1, f1 = peaks[i]

        for j in range(1, fan_value + 1):
            if i + j < len(peaks):
                t2, f2 = peaks[i + j]

                delta_t = t2 - t1
                if 0 < delta_t <= 5.0:  # Limit max time delta for compact hashes
                    # Create hash string
                    hash_str = f"{int(f1)}|{int(f2)}|{int(delta_t * 100)}"
                    h = hashlib.sha1(hash_str.encode('utf-8')).hexdigest()[0:20]  # Shorten for space
                    hashes.append((h, t1))  # Store hash + anchor time

    return hashes



'# Generate hashes\nfingerprints = generate_hashes(constellation_map)\n\n# Show some hashes\nprint("Sample Fingerprints (hash, time):")\nfor h in fingerprints[:10]:\n    print(h)'

In [69]:
import json

with open('songs_meta.json', 'r', encoding='utf-8') as f:
    songsMeta = json.load(f)

hashIndex={}
for song in songsMeta.values():
    y, sr = librosa.load(song["location"], sr=None)

    # Step 2: Compute STFT
    n_fft = 2048
    hop_length = 512
    S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    S_mag = np.abs(S)

    # Step 3: Smooth spectrogram (optional, helps with noisy peaks)
    S_smooth = gaussian_filter(S_mag, sigma=1.0)

    # Step 4: 2D local maxima detection
    neighborhood_size = 20
    local_max = maximum_filter(S_smooth, size=neighborhood_size) == S_smooth

    # Step 5: Apply a threshold to keep only strong peaks
    threshold = np.percentile(S_smooth[local_max], 95)  # keep top 5%
    peak_mask = local_max & (S_smooth >= threshold)

    # Step 6: Get peak coordinates (freq_bin, time_bin)
    peak_indices = np.argwhere(peak_mask)

    # Step 7: Convert to time (sec) and frequency (Hz)
    peak_freqs = peak_indices[:, 0] * (sr / n_fft)
    peak_times = peak_indices[:, 1] * (hop_length / sr)

    # Combine for (time, freq) tuples
    constellation_map = list(zip(peak_times, peak_freqs))

    #get fingerprints
    fingerprint = generate_hashes(constellation_map)

    for h,t in fingerprint:
        if h not in hashIndex:
            hashIndex[h]=[(song["title"],t)]
        else:
            hashIndex[h].append((song["title"], t))

C:\Users\udayg\AppData\Local\Temp\ipykernel_11228\82423531.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(song["location"], sr=None)
c:\Users\udayg\OneDrive\Desktop\NotShazam\.venv\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [47]:
def SearchSong(songLocation):
    """
    Search for a song by title and artist.
    Returns the fingerprints if found, else None.
    """
    y, sr = librosa.load(songLocation, sr=None)

    # Step 2: Compute STFT
    n_fft = 2048
    hop_length = 512
    S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    S_mag = np.abs(S)

    # Step 3: Smooth spectrogram (optional, helps with noisy peaks)
    S_smooth = gaussian_filter(S_mag, sigma=1.0)

    # Step 4: 2D local maxima detection
    neighborhood_size = 20
    local_max = maximum_filter(S_smooth, size=neighborhood_size) == S_smooth

    # Step 5: Apply a threshold to keep only strong peaks
    threshold = np.percentile(S_smooth[local_max], 95)  # keep top 5%
    peak_mask = local_max & (S_smooth >= threshold)

    # Step 6: Get peak coordinates (freq_bin, time_bin)
    peak_indices = np.argwhere(peak_mask)

    # Step 7: Convert to time (sec) and frequency (Hz)
    peak_freqs = peak_indices[:, 0] * (sr / n_fft)
    peak_times = peak_indices[:, 1] * (hop_length / sr)

    # Combine for (time, freq) tuples
    constellation_map = list(zip(peak_times, peak_freqs))

    #get fingerprints
    fingerprint = generate_hashes(constellation_map)
    score={
    }
    for h,t_query in fingerprint:
        if h in hashIndex:
            for (song,t_db) in hashIndex[h]:
                delta= round(t_db-t_query, 2)
                score[(song,delta)] = score.get((song,delta), 0) + 1
    best_match = None
    best_score = 0

    for (song, delta), count in score.items():
        if best_match is None or count > best_score:
            best_match = song
            best_score = count
    print(songsMeta[best_match])

In [77]:
SearchSong("Testsongs\\abc.mp3")

{'title': 'RAWAL x Bharg x Encore ABJ - MAGAN', 'artist': 'Rawal', 'location': 'songs\\RAWAL x Bharg x Encore ABJ - MAGAN.mp3'}


In [58]:
from yt_dlp import YoutubeDL

ydt_opts={
    'format': 'bestaudio/best',
    'extractaudio': True,  # Download only audio
    'audioformat': 'mp3',  # Save as mp3
    'outtmpl': 'songs/%(title)s.%(ext)s',  # Save to songs folder with title as filename
    'ignoreerrors': True,
    'noplaylist': False,  # Make sure it's treated as a playlist

    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}

playlist_url = 'https://www.youtube.com/playlist?list=PLwi5bnlaFJvgc7x1MPvV3h2KyU5vlVQKg'

with YoutubeDL(ydt_opts) as ydl:
    ydl.download([playlist_url])  

[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLwi5bnlaFJvgc7x1MPvV3h2KyU5vlVQKg
[youtube:tab] PLwi5bnlaFJvgc7x1MPvV3h2KyU5vlVQKg: Downloading webpage


[youtube:tab] PLwi5bnlaFJvgc7x1MPvV3h2KyU5vlVQKg: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: idk
[youtube:tab] PLwi5bnlaFJvgc7x1MPvV3h2KyU5vlVQKg page 1: Downloading API JSON
[youtube:tab] Playlist idk: Downloading 5 items of 5
[download] Downloading item 1 of 5
[youtube] Extracting URL: https://www.youtube.com/watch?v=p0kKahGOKe8
[youtube] p0kKahGOKe8: Downloading webpage
[youtube] p0kKahGOKe8: Downloading tv client config
[youtube] p0kKahGOKe8: Downloading player e12fbea4-main
[youtube] p0kKahGOKe8: Downloading tv player API JSON
[youtube] p0kKahGOKe8: Downloading ios player API JSON
[youtube] p0kKahGOKe8: Downloading m3u8 information
[info] p0kKahGOKe8: Downloading 1 format(s): 251
[download] Destination: songs\RAWAL x Bharg x Encore ABJ - MAGAN.webm
[download] 100% of    3.38MiB in 00:00:02 at 1.68MiB/s   
[ExtractAudio] Destination: songs\RAWAL x Bharg x Encore ABJ - MAGAN.mp3
Deleting original file songs\RAWAL x Bharg x Encore ABJ - M

         player = https://www.youtube.com/s/player/e12fbea4/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] 2k7E3u36gt4: Downloading m3u8 information
[info] Testing format 234
[info] 2k7E3u36gt4: Downloading 1 format(s): 234
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 46
[download] Destination: songs\Khoya Sitara - Dizlaw (Official music video) ｜ Azadi Records.mp4
[download] 100% of    3.88MiB in 00:00:22 at 177.76KiB/s                
[ExtractAudio] Destination: songs\Khoya Sitara - Dizlaw (Official music video) ｜ Azadi Records.mp3
Deleting original file songs\Khoya Sitara - Dizlaw (Official music video) ｜ Azadi Records.mp4 (pass -k to keep)
[download] Downloading item 3 of 5
[youtube] Extracting URL: https://www.youtube.com/watch?v=6Iox2oVm1sI
[youtube] 6Iox2oVm1sI: Downloading webpage
[youtube] 6Iox2oVm1sI: Downloading tv client config
[youtube] 6Iox2oVm1sI: Downloading tv player API JSON
[youtube] 6Iox2oVm1sI: Downloading ios player API JSON


         player = https://www.youtube.com/s/player/e12fbea4/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] 6Iox2oVm1sI: Downloading m3u8 information
[info] Testing format 234
[info] 6Iox2oVm1sI: Downloading 1 format(s): 234
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 50
[download] Destination: songs\Bharg & @ChaarDiwaari  - Roshni ｜ Nikamma.mp4
[download] 100% of    3.95MiB in 00:00:07 at 507.57KiB/s                
[ExtractAudio] Destination: songs\Bharg & @ChaarDiwaari  - Roshni ｜ Nikamma.mp3
Deleting original file songs\Bharg & @ChaarDiwaari  - Roshni ｜ Nikamma.mp4 (pass -k to keep)
[download] Downloading item 4 of 5
[youtube] Extracting URL: https://www.youtube.com/watch?v=jj3AOSCEGp4
[youtube] jj3AOSCEGp4: Downloading webpage
[youtube] jj3AOSCEGp4: Downloading tv client config
[youtube] jj3AOSCEGp4: Downloading tv player API JSON
[youtube] jj3AOSCEGp4: Downloading ios player API JSON


         player = https://www.youtube.com/s/player/e12fbea4/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] jj3AOSCEGp4: Downloading m3u8 information
[info] Testing format 234
[info] jj3AOSCEGp4: Downloading 1 format(s): 234
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 41
[download] Destination: songs\Naksha.mp4
[download] 100% of    3.21MiB in 00:00:16 at 195.13KiB/s                
[ExtractAudio] Destination: songs\Naksha.mp3
Deleting original file songs\Naksha.mp4 (pass -k to keep)
[download] Downloading item 5 of 5
[youtube] Extracting URL: https://www.youtube.com/watch?v=6pQcD3xKAtA
[youtube] 6pQcD3xKAtA: Downloading webpage
[youtube] 6pQcD3xKAtA: Downloading tv client config
[youtube] 6pQcD3xKAtA: Downloading tv player API JSON
[youtube] 6pQcD3xKAtA: Downloading ios player API JSON


ERROR: [youtube] 6pQcD3xKAtA: Video unavailable. This video is not available


[download] Finished downloading playlist: idk


In [ ]:
import os
import json
import re
from yt_dlp import YoutubeDL

# Step 1: Create a valid filename function
def clean_filename(name):
    return re.sub(r'[\\/*?:"<>|]', "", name)

# Step 2: Prepare metadata dictionary
songs_meta = {}

# Load existing metadata if present
meta_file = "songs_meta.json"
if os.path.exists(meta_file):
    with open(meta_file, "r", encoding="utf-8") as f:
        songs_meta = json.load(f)

# Step 3: Define output directory
output_dir = "songs"
os.makedirs(output_dir, exist_ok=True)

# Step 4: Define progress hook
def progress_hook(d):
    if d['status'] == 'finished':
        info = d['info_dict']
        title = info.get('title', 'Unknown Title')
        artist = info.get('uploader', 'Unknown Artist')
        video_id = info.get('id')
        ext = info.get('ext', 'mp3')

        safe_title = clean_filename(title)
        filename = f"{safe_title}.mp3"
        original_path = f"{video_id}.{ext}"
        final_path = os.path.join(output_dir, filename)

        if os.path.exists(original_path):
            os.rename(original_path, final_path)

        # Add or update song metadata
        songs_meta[safe_title] = {
            "title": title,
            "artist": artist,
            "location": final_path
        }

# Step 5: Define YT-DLP options
def get_yt_dlp_opts():
    return {
        'format': 'bestaudio/best',
        'ignoreerrors': True,
        'quiet': False,
        'noplaylist': False,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'progress_hooks': [progress_hook],
        'outtmpl': '%(id)s.%(ext)s',
    }

# Step 6: Start download
playlist_url = "https://www.youtube.com/playlist?list=PLwi5bnlaFJvgc7x1MPvV3h2KyU5vlVQKg"

with YoutubeDL(get_yt_dlp_opts()) as ydl:
    ydl.download([playlist_url])

# Step 7: Save updated metadata
with open(meta_file, "w", encoding="utf-8") as f:
    json.dump(songs_meta, f, indent=4, ensure_ascii=False)


[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLwi5bnlaFJvgc7x1MPvV3h2KyU5vlVQKg
[youtube:tab] PLwi5bnlaFJvgc7x1MPvV3h2KyU5vlVQKg: Downloading webpage


[youtube:tab] PLwi5bnlaFJvgc7x1MPvV3h2KyU5vlVQKg: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: idk
[youtube:tab] PLwi5bnlaFJvgc7x1MPvV3h2KyU5vlVQKg page 1: Downloading API JSON
[youtube:tab] Playlist idk: Downloading 5 items of 5
[download] Downloading item 1 of 5
[youtube] Extracting URL: https://www.youtube.com/watch?v=p0kKahGOKe8
[youtube] p0kKahGOKe8: Downloading webpage
[youtube] p0kKahGOKe8: Downloading tv client config
[youtube] p0kKahGOKe8: Downloading tv player API JSON
[youtube] p0kKahGOKe8: Downloading ios player API JSON
[youtube] p0kKahGOKe8: Downloading m3u8 information
[info] p0kKahGOKe8: Downloading 1 format(s): 251
[download] Destination: p0kKahGOKe8.webm
[download] 100% of    3.38MiB in 00:00:00 at 3.39MiB/s   


ERROR: WARNING: unable to obtain file audio codec with ffprobe


[download] Downloading item 2 of 5
[youtube] Extracting URL: https://www.youtube.com/watch?v=2k7E3u36gt4
[youtube] 2k7E3u36gt4: Downloading webpage
[youtube] 2k7E3u36gt4: Downloading tv client config
[youtube] 2k7E3u36gt4: Downloading tv player API JSON
[youtube] 2k7E3u36gt4: Downloading ios player API JSON
[youtube] 2k7E3u36gt4: Downloading m3u8 information
[info] 2k7E3u36gt4: Downloading 1 format(s): 251
[download] Destination: 2k7E3u36gt4.webm
[download] 100% of    4.17MiB in 00:00:01 at 3.18MiB/s   


ERROR: WARNING: unable to obtain file audio codec with ffprobe


...

[download] Downloading item 4 of 5
[youtube] Extracting URL: https://www.youtube.com/watch?v=jj3AOSCEGp4
[youtube] jj3AOSCEGp4: Downloading webpage
[youtube] jj3AOSCEGp4: Downloading tv client config
[youtube] jj3AOSCEGp4: Downloading tv player API JSON
[youtube] jj3AOSCEGp4: Downloading ios player API JSON
[youtube] jj3AOSCEGp4: Downloading m3u8 information
[info] jj3AOSCEGp4: Downloading 1 format(s): 251
[download] Destination: jj3AOSCEGp4.webm
[download] 100% of    3.57MiB in 00:00:00 at 4.13MiB/s   


ERROR: WARNING: unable to obtain file audio codec with ffprobe


[download] Downloading item 5 of 5
[youtube] Extracting URL: https://www.youtube.com/watch?v=6pQcD3xKAtA
[youtube] 6pQcD3xKAtA: Downloading webpage
[youtube] 6pQcD3xKAtA: Downloading tv client config
[youtube] 6pQcD3xKAtA: Downloading tv player API JSON
[youtube] 6pQcD3xKAtA: Downloading ios player API JSON


ERROR: [youtube] 6pQcD3xKAtA: Video unavailable. This video is not available


[download] Finished downloading playlist: idk
